In [5]:
import pandas as pd

In [6]:
df_weer = pd.read_excel('./weerdata.xlsx', parse_dates=['Datum'], usecols=['Datum', 'Tmax', 'Tmin', 'Neerslag', 'Max Windstoot'])

df_weer.shape

(907, 5)

In [7]:
df_bar = pd.read_csv('./data_cleaned/merged_cleaned_data.csv', parse_dates=['Datum'], dayfirst=True, usecols=['Datum', 'Aantal * prijs'])
df_bar = df_bar.rename(columns={"Aantal * prijs": 'Omzet'})

df_bar.shape

(1297232, 2)

In [8]:
# Group df_bar by hour to prevent too many combinations of rows when merged
df_bar['Datum_uur'] = df_bar['Datum'].dt.floor('h')
df_hourly = df_bar.groupby('Datum_uur')['Omzet'].sum().reset_index()
df_hourly['Datum'] = pd.to_datetime(df_hourly['Datum_uur'].dt.date)

df_hourly.shape

(3401, 3)

In [9]:
df_merged_omzet_weer = pd.merge(df_hourly, df_weer, on='Datum', how='left')

In [10]:
df_ticket = pd.read_csv('./data_output/ticket_concat_goed.csv', parse_dates=['first_event_date_start', 'last_event_date_end'])

df_ticket.head()

,event_category,first_event_date_start,last_event_date_end,first_scanned_at
0,festival,2024-07-28 14:00:00,2024-07-28 23:00:00,NaN
1,festival,2024-07-28 14:00:00,2024-07-28 23:00:00,NaN
2,festival,2024-07-28 14:00:00,2024-07-28 23:00:00,NaN
3,festival,2024-07-28 14:00:00,2024-07-28 23:00:00,NaN
4,festival,2024-07-28 14:00:00,2024-07-28 23:00:00,NaN


In [16]:
df_ticket.shape

(252476, 4)

In [11]:
df_ticket_grouped = (df_ticket.groupby('first_event_date_start').agg(aantal_tickets=('first_event_date_start', 'size'), aantal_opgedaagd=('first_scanned_at', lambda x: x.notna().sum())).reset_index())

df_ticket_grouped['Datum'] = pd.to_datetime(df_ticket_grouped['first_event_date_start'].dt.date)

df_ticket_grouped.head()

,first_event_date_start,aantal_tickets,aantal_opgedaagd,Datum
0,2022-02-26 14:00:00,605,425,2022-02-26
1,2022-03-05 16:00:00,1785,787,2022-03-05
2,2022-03-06 14:00:00,156,100,2022-03-06
3,2022-03-12 16:00:00,481,420,2022-03-12
4,2022-03-13 14:00:00,140,94,2022-03-13


In [15]:
df_ticket_grouped.shape

(230, 4)

In [12]:
df_merged_omzet_weer_ticket = pd.merge(df_merged_omzet_weer, df_ticket_grouped, on='Datum', how='left')

df_merged_omzet_weer_ticket = df_merged_omzet_weer_ticket[df_merged_omzet_weer_ticket['first_event_date_start'].notnull()]

df_merged_omzet_weer_ticket.head()

,Datum_uur,Omzet,Datum,Tmax,Tmin,Neerslag,Max Windstoot,first_event_date_start,aantal_tickets,aantal_opgedaagd
28,2022-07-09 14:00:00,9.00,2022-07-09,21.6 °C,15.3 °C,"0,0 mm",30.6 km/u,2022-07-09 16:00:00,1319.0,962.0
29,2022-07-09 16:00:00,182.35,2022-07-09,21.6 °C,15.3 °C,"0,0 mm",30.6 km/u,2022-07-09 16:00:00,1319.0,962.0
30,2022-07-09 17:00:00,767.10,2022-07-09,21.6 °C,15.3 °C,"0,0 mm",30.6 km/u,2022-07-09 16:00:00,1319.0,962.0
31,2022-07-09 18:00:00,1933.70,2022-07-09,21.6 °C,15.3 °C,"0,0 mm",30.6 km/u,2022-07-09 16:00:00,1319.0,962.0
32,2022-07-09 19:00:00,2567.40,2022-07-09,21.6 °C,15.3 °C,"0,0 mm",30.6 km/u,2022-07-09 16:00:00,1319.0,962.0


In [13]:
df_merged_omzet_weer_ticket.shape

(2103, 10)

In [14]:
df_merged_omzet_weer_ticket.to_csv('./merged_omzet_weer_ticket.csv')